## Tutorial 6: Adding stimulation to the network
* http://doc.netpyne.org/tutorial.html#tutorial-6-adding-stimulation-to-the-network

* 'stimSourceParams' to define the parameters of the sources of stimulation
* 'stimTargetParams' to specify what cells will be applied what source of stimulation (mapping of sources to cells).

* Same neural network as in Tutorial 2, but with stimulation added to the network.
* and Connectivity parameters are removed.


In [151]:
# load library
from netpyne import specs, sim

# Network parameters
netParams = (
    specs.NetParams()
)  # object of class NetParams to store the network parameters

In [152]:
# For cell parameters,
# "secs" is a list of dictionaries, each containing the parameters of a section.
# e.g.: gemoetry, mechanisms, etc.
PYRcell = {"secs": {}}
PYRcell["secs"]["soma"] = {"geom": {}, "mechs": {}}
PYRcell["secs"]["soma"]["geom"] = {
    "diam": 18.8,
    "L": 18.8,
    "Ra": 123.0,
}  # soma geometry
PYRcell["secs"]["soma"]["mechs"]["hh"] = {
    "gnabar": 0.12,
    "gkbar": 0.036,
    "gl": 0.003,
    "el": -70,
}  # soma hh mechanism
netParams.cellParams["PYR"] = PYRcell

# one line version
# netParams.cellParams['PYR'] = {
#    'secs': {'soma':
#            {'geom': {'diam': 18.8, 'L': 18.8, 'Ra': 123.0},
#            'mechs': {'hh': {'gnabar': 0.12, 'gkbar': 0.036, 'gl': 0.003, 'el': -70}}}}})

In [153]:
## Population parameters
netParams.popParams["S"] = {"cellType": "PYR", "numCells": 20}
netParams.popParams["M"] = {"cellType": "PYR", "numCells": 20}

In [154]:
## Synaptic mechanism parameters
netParams.synMechParams["exc"] = {
    "mod": "Exp2Syn",
    "tau1": 0.1,  # rise time
    "tau2": 5.0,  # decay time
    "e": 0,  # reversal potential
}  # excitatory synaptic mechanism

In [155]:
netParams.stimSourceParams["Input_1"] = {
    "type": "IClamp",
    "del": 300,  # ms, start time
    "dur": 100,  # ms, duration
    "amp": "uniform(0.4,0.5)",  # nA, amplitude
}
netParams.stimSourceParams["Input_2"] = {
    "type": "VClamp",
    "dur": [100, 100],  # ms, duration
    "amp": [-70, 0],  # mV, amplitude
    "gain": 1e5,
    "rstim": 1,
    "tau1": 1.0,
    "tau2": 0,
}
netParams.stimSourceParams["Input_3"] = {
    "type": "AlphaSynapse",
    "onset": "uniform(300,600)",  # ms, start time, uniform distribution
    "tau": 5,
    "gmax": "post_ynorm",  # nS, maximum conductance, target cell normalized depth
    "e": 0,
}
netParams.stimSourceParams["Input_4"] = {
    "type": "NetStim",
    "interval": "uniform(20,100)",
    "number": 1000,
    "start": 600,
    "noise": 0.1,
}

In [156]:
netParams.stimTargetParams["Input_1->S"] = {
    "source": "Input_1",
    "sec": "soma",
    "loc": 0.8,
    "conds": {"pop": "S", "cellList": range(15)},
}
netParams.stimTargetParams["Input_2->S"] = {
    "source": "Input_2",
    "sec": "soma",
    "loc": 0.5,
    "conds": {"pop": "S", "ynorm": [0, 0.5]},
}

netParams.stimTargetParams["Input_2->M"] = {
    "source": "Input_2",
    "sec": "soma",
    "loc": 0.5,
    "conds": {"pop": "M", "cellList": [0]},
}


netParams.stimTargetParams["Input_3->M1"] = {
    "source": "Input_3",
    "sec": "soma",
    "loc": 0.2,
    "conds": {"pop": "M", "cellList": [2, 4, 5, 8, 10, 15, 19]},
}
netParams.stimTargetParams["Input_4->PYR"] = {
    "source": "Input_4",
    "sec": "soma",
    "loc": 0.5,
    "weight": "0.1+normal(0.2,0.05)",
    "delay": 1,
    "conds": {"cellType": "PYR", "ynorm": [0.6, 1.0]},
}

In [157]:
## Simulation options
simConfig = (
    specs.SimConfig()
)  # object of class SimConfig to store simulation configuration

simConfig.duration = 1 * 1e3  # Duration of the simulation, in ms
simConfig.dt = 0.025  # Internal integration timestep to use
simConfig.verbose = False  # Show detailed messages
simConfig.recordTraces = {
    "V_soma": {"sec": "soma", "loc": 0.5, "var": "v"},
    "I_Na_soma": {"sec": "soma", "loc": 0.5, "var": "ina"},
}  # Dict with traces to record
simConfig.recordStep = 0.1  # Step size in ms to save data (eg. V traces, LFP, etc)
simConfig.filename = "./results/Tutorial06/tut6"  # Set file output name
simConfig.savePickle = False  # Save params, network and sim output to pickle file

simConfig.analysis["plotRaster"] = {"saveFig": True}  # Plot a raster
simConfig.analysis["plotTraces"] = {
    "include": [1, 18, 19, 20, 39],
    "saveFig": True,
}  # Plot recorded traces for this list of cells
simConfig.analysis["plot2Dnet"] = {
    "saveFig": True
}  # plot 2D visualization of cell positions and connections

In [158]:
## Create network and run simulation
sim.createSimulateAnalyze(netParams, simConfig)


Start time:  2024-10-29 12:33:22.356663




Creating network of 2 cell populations on 1 hosts...: 100%|##########|

  Number of cells on node 0: 40 
  Done; cell creation time = 0.02 s.
Making connections...
  Number of connections on node 0: 0 
  Done; cell connection time = 0.00 s.
Adding stims...
  Number of stims on node 0: 49 
  Done; cell stims creation time = 0.01 s.
Recording 10 traces of 2 types on node 0

Running simulation using NEURON for 1000.0 ms...
  Done; run time = 0.67 s; real-time ratio: 1.48.

Gathering data...
  Done; gather time = 0.01 s.

Analyzing...
  Cells: 40
  Connections: 16 (0.40 per cell)
  Spikes: 125 (3.12 Hz)
  Simulated time: 1.0 s; 1 workers
  Run time: 0.67 s
Preparing spike data...
Plotting raster...



exp(955.401) out of range, returning exp(700)
exp(955.901) out of range, returning exp(700)
exp(953.901) out of range, returning exp(700)
exp(1392.5) out of range, returning exp(700)
No more errno warnings during this execution


Plotting recorded cell traces ... cell
Plotting 2D representation of network cell locations and connections...
  Done; plotting time = 1.08 s

Total time = 1.79 s
